In [ ]:
import requests 
import json 
from bs4 import BeautifulSoup
import sys

import pandas as pd 
import numpy as np 
import scipy.stats as sci 

import datetime  
import time 
import random 

import warnings
warnings.filterwarnings(action = 'ignore', category = FutureWarning) 

In [ ]:
steamLoginSecure = '76561198329918048%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MEJGQ18yMTgyMTQ5N182MjQwNCIsICJzdWIiOiAiNzY1NjExOTgzMjk5MTgwNDgiLCAiYXVkIjogWyAid2ViIiBdLCAiZXhwIjogMTY2OTc5ODQwNiwgIm5iZiI6IDE2NjEwNzA2MTIsICJpYXQiOiAxNjY5NzEwNjEyLCAianRpIjogIjBDNjRfMjFBRUM5NTBfQUU4MEUiLCAib2F0IjogMTY2NzAxMTc3NywgInJ0X2V4cCI6IDE2ODQ4NzY5NzUsICJwZXIiOiAwLCAiaXBfc3ViamVjdCI6ICIxMjUuMjM1LjIzOS4xNyIsICJpcF9jb25maXJtZXIiOiAiMTI1LjIzNS4yMzkuMTciIH0.4Yo3Ur2HcAw2FplCBgiWiaehPrpqVcelCa0WUxLEQImcDsyPC3Xi-KLenMG7CWjn-oVQb7dJclU8b5I6e3yoAQ'
cookie = {'steamLoginSecure': steamLoginSecure}

In [ ]:
URL = 'https://steamcommunity.com/market/search/render/?query=&start=0&count=100&search_descriptions=0&sort_column=price&sort_dir=desc&appid=730&norender=1'
Items_get = requests.get(url = URL, cookies = cookie)
Items = json.loads(Items_get.text)

data = Items['results']
total_Items = Items['total_count']


In [ ]:
print(total_Items)

In [ ]:
Items_list = []

In [ ]:
# get all items information, because steam only allow 100 items/request, and ~20 requests/minute so everytime request return code 400, put to sleep for 70 seconds
pos = 0
while (pos < total_Items) :
    while (True) :
        URL = 'https://steamcommunity.com/market/search/render/?query=&start=' + str(pos) + '&count=100&search_descriptions=0&sort_column=price&sort_dir=desc&appid=730&norender=1'
        Items_get = requests.get(url = URL, cookies = cookie)
        if(Items_get.status_code != 200) :
            time.sleep(70)
            break
        
        Items = json.loads(Items_get.text)
        data = Items['results']
        for item in data :
            Items_list.append(item['asset_description'])
            
        pos += 100

In [ ]:
Item_extra_list = ['StatTrak™', 'Souvenir', 'Normal']

Item_type_list = ['Rifle', 'Sniper Rifle', 'Pistol', 'SMG', 'Shotgun', 'Machinegun', 
                  'Knife', 'Gloves', 'Sticker', 'Container', 'Others']

Exterior_list = ['Battle-Scarred', 'Well-Worn', 'Field-Tested', 'Minimal Wear', 'Factory New', 'None']

Rarity_list = ['Base Grade', 'Consumer Grade', 'Industrial Grade', 'Mil-Spec', 'Restricted', 
               'Classified', 'Covert', 'Extraordinary',
                  'High Grade', 'Remarkable', 'Exotic', 'None']

def _get_type(item_info) :                # get type of an item
    for cur_type in Item_type_list :
        if (item_info.find(cur_type) != -1) : return cur_type
    
    return Item_type_list[-1]

def _get_extra(item_info) :              # get extra status of an item
    for cur_extra in Item_extra_list :
        if (item_info.find(cur_extra) != -1) : return cur_extra
    
    return Item_extra_list[-1]

def _get_rarity(item_info) :              # get rarity of an item
    for cur_rarity in Rarity_list :
        if (item_info.find(cur_rarity) != -1) : return cur_rarity
    
    return Rarity_list[-1]

def _get_exterior(item_description) :     # get exterior of an item
    for cur_exterior in Exterior_list :
        if (item_description.find(cur_exterior) != -1) : return cur_exterior
    
    return Exterior_list[-1]



In [ ]:
def _format_ASCII_Name(name) :            # format special characters into ASCII letter
    format_name = ''
    for ch in name :
        if (ch == '|')   : ch = '%7C'
        elif (ch == '&') : ch = '%26'
        elif (ch == ':') : ch = '%3A'
        elif (ch == '/') : ch = '%2F'
        elif (ch == '+') : ch = '%2B'
        elif (ch == ' ') : ch = '%20'
        elif (ch == '(') : ch = '%28'
        elif (ch == ')') : ch = '%29'
        
        format_name += ch
            
    return format_name

In [ ]:
def _get_price_request(name) :        # get price data from a skin
    while (True) :
        URL             = 'http://steamcommunity.com/market/pricehistory/?country=PT&currency=3&appid=730&market_hash_name=' + format_name
        item_price_data = requests.get(url = URL, cookies = cookie);
        if (item_price_data.status_code != 200) :
            #print(name, item_price_data.status_code)
            continue
            
        return json.loads(item_price_data.text)
        

In [ ]:
items_df = pd.DataFrame(data = None, index = None, columns = ['Name', 'Type', 'Exterior', 'Rarity', 'Extra', 'Average volume sale',
                                                              'Average price (VND)', 'Max price (VND)', 'Days to reach max', 
                                                              'Min price (VND)', 'Days to reach min', 'Price change (%)',
                                                                  'Time on market (days)'])

cnt = 0
for cur_item in Items_list :        # Loop through each item in Item_list
    item_name        = cur_item['market_hash_name']           # get item's name
    item_info        = cur_item['type']                       # get item's information
    item_description = cur_item['descriptions'][0]['value']   # get text contain item's exterior
    
    item_exterior = _get_exterior(item_description)     # get exterior
    item_rarity   = _get_rarity(item_info)              # get rarity
    item_extra    = _get_extra(item_info)               # get extra
    item_type     = _get_type(item_info)                # get type
    
    format_name     = _format_ASCII_Name(item_name)     # format name into ASCII format
    item_price_data = _get_price_request(format_name)   # send request to get price data
    
    if (item_price_data) :                              # if we got information about the item then proceed
        item_prices = item_price_data['prices']         
        success = item_price_data['success']
        if (item_prices == '') | (not item_prices) | (success == False): continue
        
        price_list  = []            # store price each day
        vol_list    = []            # store volume sales each day
        dates       = []            # store dates
        
        tmp_price_list = []         # temp price list
        tmp_vol_list   = []         # temp vol list
        
        last_date = datetime.date(1, 1, 1)
        
        l, r = 0, 0
        
        # There could be data of multiple sales of the same day
        # So we calculate price for a day is the average of all price sale in that day, using tmp_price_list
        # same for vol_list
        
#         print(cnt, item_name)
#         cnt += 1
        
        while (r <= len(item_prices)) :
            cur_date = datetime.date(1, 1, 1)
            if (r < len(item_prices)) :            # still in list's boundary
                cur_date = datetime.datetime.strptime(item_prices[r][0][0 : 11], '%b %d %Y').date()
                
            if (cur_date != last_date and r != 0) :
                avg_price = np.mean(tmp_price_list[l : r])    # get average price of a day
                avg_vol   = np.mean(tmp_vol_list[l : r])      # get average volume sales of a day
                
                price_list.append(avg_price)
                vol_list.append(avg_vol)
                dates.append(last_date)
                
                l = r
            
            if (r < len(item_prices)) : 
                tmp_price_list.append(item_prices[r][1])
                tmp_vol_list.append(float(item_prices[r][2]))
            
            
            last_date = cur_date
            r += 1
        
        avg_price = np.mean(price_list)           # get average price of the item
        avg_vol   = np.mean(vol_list)             # get average sale volumes of the item
        
        max_price = max(price_list)           # get max price of the item 
        min_price = min(price_list)           # get min price of the item
        
        max_idx     = price_list.index(max_price)
        days_to_max = (dates[max_idx] - dates[0]).days   # calculate days to reach maximum
        
        min_idx     = price_list.index(min_price)
        days_to_min = (dates[min_idx] - dates[0]).days   # calculate days to reach minimum
        
        if (price_list[0] == 0) : price_list[0] = 1
        price_change = (price_list[-1] - price_list[0]) / price_list[0]       # calculate percentage of price change from start till now
        price_change = round(price_change, 2)
                                  
        time_on_market   = (dates[-1] - dates[0]).days           # calculate the total days the item has existed
        
        item_dict = {'Name' : item_name, 'Type' : item_type, 'Exterior' : item_exterior, 'Rarity' : item_rarity,
                      'Extra' : item_extra, 'Average volume sale' : avg_vol, 'Average price (VND)' : avg_price,
                        'Max price (VND)' : max_price, 'Days to reach max' : days_to_max,  'Min price (VND)' : min_price, 
                         'Days to reach min' : days_to_min, 'Price change (%)' : price_change, 'Time on market (days)' : time_on_market}
        
        cur_item_df = pd.DataFrame(item_dict, index = [0]);
        items_df    = items_df.append(cur_item_df, ignore_index = True)  
  

#items_df    

In [ ]:
items_df.to_csv('CSGO_Market.csv', index = False)

In [ ]:
tmp_df = pd.read_csv('CSGO_Market.csv')
tmp_df